<a href="https://colab.research.google.com/github/Tanguyvans/HackAI/blob/main/Compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Quantization**

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat May 18 13:30:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [4]:
import sys
import os
import numpy as np
from google.colab.patches import cv2_imshow
import cv2
from IPython.display import Image

import torch
import torch
import numpy as np
import torch
from tqdm.auto import tqdm
assert torch.cuda.is_available()

In [2]:

!pip install torch2trt
#from torch2trt import torch2trt

ERROR: Could not find a version that satisfies the requirement torch2trt (from versions: none)
ERROR: No matching distribution found for torch2trt


In [5]:
import os
os.environ["os1"]="ubuntu1804"
os.environ["tag"]= "cuda11.0-trt7.2.3.4-ga-20210226" #@param
os.environ["version"]= "7.2.3-1+cuda11.0" #@param
data_path = '/content/drive/MyDrive/IA/UMONS/Workshop/Compression' #@param
os.chdir(data_path)


In [ ]:
!sudo dpkg -i nv-tensorrt-repo-{tag}_1-1_amd64.deb
!sudo apt-key add /var/nv-tensorrt-repo-{version} libnvonnxparsers7={version} libnvinfer-plugin7={version} libnvonnxparsers-dev={version} libnvinfer-plugin-dev={version} python3-libnvinfer={version}
!sudo apt-get install python3-libnvinfer-dev=${version}

!git clone https://github.com/NVIDIA-AI-IOT/torch2trt /content/torch2trt
%cd /content/torch2trt
!python setup.py install

In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [3]:
folder = '/content/drive/MyDrive/IA/UMONS/Workshop/' #@param ['/content/drive/MyDrive/Workshop/', '/content/drive/MyDrive/IA/UMONS/Workshop/']

#classification_path = f"{folder}2. Image classification/" #@param
classif_folder= '2. Image classification/' #@param
classification_path = folder + classif_folder
sys.path.append(classification_path)
os.chdir(classification_path)


NameError: name 'sys' is not defined

In [ ]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor, Compose, Normalize, Resize
class ImageFolderCalibDataset():
    def __init__(self, root):
        self.dataset = ImageFolder(
            root=root,
            transform=Compose([
                Resize((224, 224)),
                ToTensor(),
                Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        )
    def __len__(self):
        return len(self.dataset)
    def __getitem__(self, idx):
        image, _ = self.dataset[idx]
        image = image[None, ...]  # add batch dimension
        return [image]

In [ ]:
def Bit_Compressor(model_path,fp16_mode=True,int8_mode=True,calibration=True,images_folder):
  import sys
  import torch
  import numpy as np
  from tqdm.auto import tqdm
  assert torch.cuda.is_available()
  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  model = torch.load(model_path).to(device)
  example_input = torch.randn(1, 3, 224, 224).cuda()
  #if mode='fp16':
    #model_trt = torch2trt(model,[example_input], fp16_mode=True)
  #if mode='int8':
    #model_trt = torch2trt(model,[example_input], int8_mode=True)
  if calibration==True:
    dataset = ImageFolderCalibDataset(images_folder)
    model_trt = torch2trt(model,[example_input], fp16_mode=fp16_mode,int8_mode=int8_mode,int8_calib_dataset=dataset)
  else:
    model_trt = torch2trt(model,[example_input], fp16_mode=fp16_mode,int8_mode=int8_mode)
  #torch.save(model_trt.state_dict(), model_path+'_quantified')
  return model_trt

NameError: name 'torch2trt' is not defined

# **Teacher-Student**

In [ ]:
from torchvision import transforms

import sys,os
import torch, torchvision
#import torchvision.transforms as transforms
from torchvision import datasets, transforms
import torch.nn as nn
from torchsummary import summary

from torch import Tensor
from functools import partial

from torchvision.transforms._presets import ImageClassification
import torchvision.models as models
from torchvision.models import ResNet50_Weights
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

In [ ]:
image_classification_path = f'{folder}2. Image classification'

training_path=   image_classification_path + '/' + 'data'
testing_path = image_classification_path + '/' + 'test'
sys.path.append(image_classification_path)
os.chdir(image_classification_path)
print(image_classification_path)

In [ ]:
batch_size="32" #@param [16,32]
batch_size = int(batch_size)
validation_split = 0.1
seed = 50 #@param

In [68]:
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import copy
import os
import torch.nn.functional as F
import random
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, RandomHorizontalFlip

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Evaluation loop
@torch.no_grad()
def evaluate(
    model: nn.Module,
    dataloader: DataLoader,
    verbose=True,
) -> float:
    model.eval()

    num_samples = 0
    num_correct = 0
    loss = 0

    for inputs, targets in tqdm(dataloader, desc="eval", leave=False, disable=not verbose):
        # Move the data from CPU to GPU
        inputs = inputs.to(device)
        targets = targets.to(device)
        # Inference
        outputs = model(inputs)
        # Calculate loss
        loss += F.cross_entropy(outputs, targets, reduction="sum")
        # Convert logits to class indices
        outputs = outputs.argmax(dim=1)
        # Update metrics
        num_samples += targets.size(0)
        num_correct += (outputs == targets).sum()
    return (num_correct / num_samples * 100).item(), (loss / num_samples).item()

# training loop
def train_kd(
    model_student: nn.Module,
    path_teacher,
    train_loader: DataLoader,
    test_loader: DataLoader,
    epochs: int,
    lr: int,
    path,
    weight_decay=5e-4,
    callbacks=None,
    save=None,
    save_only_state_dict=False,

) -> None:

    print('train student model')
    model_teacher = torch.load(path_teacher)

    optimizer = torch.optim.SGD(model_student.parameters(
    ), lr=lr, momentum=0.9, weight_decay=weight_decay)
    # scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[40,80], gamma=0.1)
    criterion = nn.CrossEntropyLoss()
    best_acc = -1
    best_checkpoint = dict()


    for epoch in range(epochs):
        model_student.train()
        model_teacher.train()
        for inputs, targets in tqdm(train_loader, leave=False):
            inputs = inputs.to(device)
            targets = targets.to(device)

            # Reset the gradients (from the last iteration)
            optimizer.zero_grad()

            # Forward inference
            out_student = model_student(inputs)
            out_teacher = model_teacher(inputs)


            # kd loss
            kd_T = 4
            predict_student = F.log_softmax(out_student / kd_T, dim=1)
            predict_teacher = F.softmax(out_teacher / kd_T, dim=1)
            alpha = 0.9
            loss = nn.KLDivLoss()(predict_student, predict_teacher) * (alpha * kd_T * kd_T) + criterion(out_student, targets) * (1-alpha)

            loss.backward()


            # Update optimizer
            optimizer.step()

            if callbacks is not None:
                for callback in callbacks:
                    callback()

        acc, val_loss = evaluate(model_student, test_loader)
        print(
            f'Epoch {epoch + 1}/{epochs} | Val acc: {acc:.2f} | Val loss: {val_loss:.4f} | LR: {optimizer.param_groups[0]["lr"]:.6f}')

        if best_acc < acc:
            best_checkpoint['state_dict'] = copy.deepcopy(model_student.state_dict())
            best_acc = acc
        # Update LR scheduler
        scheduler.step()
    model_student.load_state_dict(best_checkpoint['state_dict'])
    if save:
        # on veut sauvegarder le meilleur modèle
        #path = os.path.join(os.getcwd(), "results", save)
        #os.makedirs(os.path.dirname(path), exist_ok=True)
        #path=os.getcwd()
        if save_only_state_dict:
            torch.save(model_student.state_dict(), path)
        else:
            torch.save(model_student, path)
    print(f'Best val acc: {best_acc:.2f}')

# Fixer le seed pour la reproductibilité
import numpy as np
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)


# train student model
def Teacher_Student(student_model,teacher_path,train_dataloader,test_dataloader,epochs=100,lr=0.001,save=True):
  print('train student model')
  teacher_model = torch.load(teacher_path)
  train_kd(student_model, teacher_model, train_dataloader, test_dataloader, epochs, lr, path,save)

In [18]:
transformView = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.RandomHorizontalFlip(),  # Randomly flip the images horizontally
    #transforms.RandomRotation(15),       # Randomly rotate the images by +/- 15 degrees
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
test_transforms = transforms.Compose([
    transforms.Resize(255),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    #transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
datasetView = datasets.ImageFolder(training_path, transformView)
#dataloaderView = torch.utils.data.DataLoader(datasetView, batch_size=batch_size, shuffle=True)
test_dataset = datasets.ImageFolder(root=testing_path, transform=test_transforms)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [19]:
from torch.utils.data import random_split
total_size = len(datasetView)
train_size = int(total_size * (1 - validation_split))
validation_size = total_size - train_size
torch.manual_seed(seed)
train_dataset, validation_dataset = random_split(datasetView, [train_size, validation_size])
# Create DataLoaders for each dataset
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

In [74]:
def create_mobilenet(num_classes):
    # Load a pre-trained MobileNetV2 model
    model = models.mobilenet_v2(pretrained=True)

    # Freeze all the parameters in the model
    for param in model.parameters():
        param.requires_grad = False

    # MobileNetV2 uses 'classifier' instead of 'fc' used by ResNet
    # The last layer of MobileNetV2's classifier is a Linear layer with 1280 input features
    num_ftrs = model.classifier[1].in_features

    # Replace the classifier with a new one
    model.classifier = nn.Sequential(
        nn.Linear(num_ftrs, 128),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(128, 32),
        nn.ReLU(),
        nn.Dropout(0.4),
        nn.Linear(32, num_classes),
        nn.Softmax(dim=1)  # Softmax for classification
    )
    return model

In [78]:
student_model = create_mobilenet(3)
student_model.to(device)
path_teacher=image_classification_path + '/' + 'model-Resnet-Epoch10.pth'
#student_model=models.mobilenet_v2(pretrained=True)
epochs=10
#f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%"
path=classification_path + '/' + f"student_mobilenet_e{epochs}_resnet_e10"
print(path)

/content/drive/MyDrive/IA/UMONS/Workshop/2. Image classification//student_mobilenet_e10_resnet_e10


In [79]:
train_kd(student_model, path_teacher, train_loader, validation_loader, epochs=epochs, lr=0.01, path=path,save=True)

train student model


Epoch 1/10 | Val acc: 50.71 | Val loss: 1.0116 | LR: 0.010000


Epoch 2/10 | Val acc: 84.36 | Val loss: 0.7875 | LR: 0.010000


Epoch 3/10 | Val acc: 84.83 | Val loss: 0.7107 | LR: 0.010000


Epoch 4/10 | Val acc: 88.63 | Val loss: 0.6721 | LR: 0.010000


...

Epoch 6/10 | Val acc: 89.57 | Val loss: 0.6586 | LR: 0.010000


Epoch 7/10 | Val acc: 90.52 | Val loss: 0.6484 | LR: 0.010000


Epoch 8/10 | Val acc: 91.00 | Val loss: 0.6490 | LR: 0.010000


Epoch 9/10 | Val acc: 91.94 | Val loss: 0.6351 | LR: 0.010000


Epoch 10/10 | Val acc: 90.52 | Val loss: 0.6409 | LR: 0.010000
Best val acc: 91.94


In [82]:
def evaluate_model(model, dataloader, criterion, device):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():  # No need to track gradients for evaluation
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total * 100
    return avg_loss, accuracy

In [80]:
student_model = torch.load(path)

In [ ]:
test_loss, test_accuracy = evaluate_model(student_model, train_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

In [83]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
test_loss, test_accuracy = evaluate_model(student_model, validation_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

Test Loss: 0.6415, Test Accuracy: 91.94%


In [84]:
import torch.nn as nn
criterion = nn.CrossEntropyLoss()
test_loss, test_accuracy = evaluate_model(student_model, train_loader, criterion, device)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

Test Loss: 0.6439, Test Accuracy: 90.54%
